# Imports

In [1]:
import pandas as pd
# import numpy as np
from datetime import datetime
import altair as alt
# import matplotlib.pyplot as plt
# import seaborn as sns

# Functions  

I will define some functions to avoid repetation in the code.

In [2]:
def raw_date_to_datetime(raw_date):
    datetime_dict = {"Date": None, "MonthName": None, "Time": None}  # New row to replace
    datetime_dict["Date"] = raw_date.strftime("%d-%m-%Y")  # Get the date in the format "DD.MM.YYYY"
    datetime_dict["Time"] = raw_date.strftime("%H:%M%z")  # Get the time in format "HH:MM+HHMM"
    datetime_dict["MonthName"] = raw_date.strftime("%B")  # Get the full month name
    datetime_dict["DayName"] = raw_date.strftime("%A")  # Get the full day name

    return datetime_dict

def ceil_dt(dt, delta):
    dt_min = datetime.min.replace(tzinfo=dt.tzinfo)
    return dt + (dt_min - dt) % delta

def is_subscribed_to(subreddit_name, subscribed_subs_df):
    return (subscribed_subs_df == subreddit_name).any().any()

# Data Collecting

Get the data from the .csv files, clear and format a bit to get useful pandas DataFrames.  

## Different Data
* Subscribed Subreddits
* IP Logs
* Post Votes
* Comment Votes
* Posts
* Comments

In [3]:
# Data path
data_path = "./data/"

## Subscribed Subreddits

This data holds the subreddits that I am _currently_ subscribed to. It does not hold a history or any temporal data.  

Reddit subreddits doesn't include direct a way to _categorize_ them by some kind of a tag or topic system execpt a list of _"flairs"_ that can be used to label posts in that subreddit. However, to better analyse them a tag system might be useful. From a list of basic tags subs can be tagged by hand. If the read data has a sub missing tags then it will automatically asks for tags from a pre-defined list of tags. After updating the tags data will be written onto the original .csv file.  

This data file also includes followed users (not subs). They are marked with a 'u_' prefix in the data. They will be filtered, but saved anyway since they might be useful in the future.

In [4]:
# Read the file, sort by name of the subreddits and reset the index after sorting
fname = "subscribed_subreddits.csv"
subreddits_df = pd.read_csv(data_path + fname).sort_values(by="subreddit").reset_index().drop(columns="index")

In [5]:
# Predefine the minimal amount of basic and useful tags
predefined_tags = tuple(sorted((
    'meme',
    'movie/show',
    'fandom',
    'game',
    'hobby',
    'information',
    'programming',
    'other',
    'user',
)))

retake_tags = False
try:
    # Check if the 'Tags' column exists at all
    sub_tags = subreddits_df["Tags"]
except KeyError:
    # Add an empty 'Tags' column
    subreddits_df["Tags"] = None
    sub_tags = subreddits_df["Tags"]
    retake_tags = True

# Check if there are any missing entry without tags
if sub_tags.isnull().sum() > 0:
    retake_tags = True

# Ask for tags
if retake_tags:
    save = False
    for idx in range(len(sub_tags)):
        sub = subreddits_df.loc[idx, "subreddit"]
        tags = subreddits_df.loc[idx, "Tags"]
        if type(tags) != str:
            tags = list()
            while True:
                tag = input(f"Enter a tag for the subreddit '{sub}', amongst {predefined_tags}\nEnter 'q' to finish.\n")
                if tag == "save":
                    subreddits_df.loc[idx, "Tags"] = ", ".join(sorted(tags))
                    save = True
                    break
                if tag == 'q':
                    if len(tags) != 0:
                        subreddits_df.loc[idx, "Tags"] = ", ".join(sorted(tags))
                        print("-----")
                        break
                    else:
                        print("You didn't enter any valid tags yet!")
                        continue
                if tag in predefined_tags and tag not in tags:
                    tags.append(tag)
                else:
                    print("You didn't enter a valid tag.")
        if save:
            break

subreddits_df.to_csv(data_path + fname, index=False)

In [6]:
followed_users_df = subreddits_df[subreddits_df["subreddit"].str.contains("u_")]
subreddits_df = subreddits_df[~subreddits_df["subreddit"].str.contains("u_")]

subreddits_df

,subreddit,Tags,Flairs
0,AskScienceFiction,information,NaN
1,CodeBullet,other,"Meme, Question For Codebullet, Video idea, Oth..."
2,DMAcademy,"game, hobby, information","Offering Advice, Need Advice: Encounters & Adv..."
3,DaystromInstitute,"fandom, information, movie/show",NaN
4,Deepspaceninememes,"fandom, meme, movie/show","Original Content [OC], Shitpost"
5,ElectroBOOM,"information, other","FAF - RECTIFY, ElectroBOOM Question, Non-Elect..."
6,ExposurePorn,"hobby, other",NaN
7,FATErpg,"game, hobby, information",NaN
8,GeekyaparLamers,other,NaN
9,GreekMythology,information,"Discussion, Question, Art, Culture, History, I..."


## IP Logs

IP logs data holds information about my logins to Reddit. It holds the date, time and the IP that I used. This data might be used on showing my active times even though it doesn't hold information on how long I have stayed active.  

The date data is in the form of "yyyy-mm-dd hh:mm:ss UTC". I will split the date and time, convert time into GMT+3, and name the months.

In [7]:
# Read the file, drop the first row that holds the registiration IP only, drop the IP column and reset the indexing
fname = "ip_logs.csv"
login_datetime_df = pd.read_csv(data_path + fname).rename(columns={"date": "RawDate"}).drop(index=0, columns="ip").reset_index().drop(columns="index")

In [8]:
try:
    raw_date_col = login_datetime_df["RawDate"]  # Raw Date column

    # Add new columns
    login_datetime_df[["Date", "Time", "MonthName", "DayName"]] = None
    
    for idx in range(len(raw_date_col)):
        raw_date = raw_date_col.iloc[idx].replace(" UTC", "")  # Get the time in UTC time
    
        # Convert datetime to local time zone
        local_datetime = datetime.fromisoformat(raw_date).astimezone()
        datetime_dict = raw_date_to_datetime(local_datetime)

        # Insert items from datetime_dict to the new columns
        for key in datetime_dict:
            login_datetime_df.loc[idx, key] = datetime_dict[key]

    login_datetime_df = login_datetime_df.drop(columns="RawDate")
except KeyError:
    pass

login_datetime_df["Date"] = pd.to_datetime(login_datetime_df["Date"], format="%d-%m-%Y")
login_datetime_df

,Date,Time,MonthName,DayName
0,2023-06-29,10:16+0300,June,Thursday
1,2023-06-29,14:25+0300,June,Thursday
2,2023-06-30,01:16+0300,June,Friday
3,2023-06-30,04:15+0300,June,Friday
4,2023-06-30,05:46+0300,June,Friday
...,...,...,...,...
357,2023-10-06,12:19+0300,October,Friday
358,2023-10-06,13:40+0300,October,Friday
359,2023-10-06,15:16+0300,October,Friday
360,2023-10-07,08:04+0300,October,Saturday


## Post Votes

This data includes the posts that I have voted. It includes an ID, the post link and the type of the vote and through Reddit API it includes the total number of upvotes and downvotes; however, no temporal data.  

I will get the subreddit name from the URL, my vote and I will compare the sub to the subscribed subs data and get wheter or not I am subscribed to that subreddit currently.  

Also, note that some of the posts are inaccesible due to different reasons which prevents data collection through Reddit API; therefore, there are some missing values in the data.

In [9]:
# Read the file, rename the vote direction column and drop the id column.
fname = "post_votes.csv"
post_votes_df = pd.read_csv(data_path + fname).rename(columns={"direction": "MyVote", "Upvotes": "UpvoteCount", "Downvotes": "DownvoteCount"}).drop(columns="id")

In [10]:
# Add the new columns
post_votes_df[["SubredditName", "IsSubscribed"]] = None

for idx in range(len(post_votes_df["permalink"])):
    post_vote_dict = {"SubredditName": None, "IsSubscribed": None}
        
    # Get the sub name from the link
    permalink = post_votes_df.loc[idx, "permalink"]
    start_idx = permalink.find("r/") + 2
    stop_idx = permalink.find("/", start_idx)
    sub_name = permalink[start_idx:stop_idx]
    
    # Add sub name to the corresponding place
    post_votes_df.loc[idx, "SubredditName"] = sub_name

    # Check if the sub is subscribed
    post_votes_df.loc[idx, "IsSubscribed"] = is_subscribed_to(sub_name, subreddits_df)

# Drop the permalink column
post_votes_df = post_votes_df.drop(columns="permalink")
# Specify the Dtypes for later use
post_votes_df["IsSubscribed"] = post_votes_df["IsSubscribed"].astype(dtype="bool")
post_votes_df

,MyVote,UpvoteCount,DownvoteCount,Flair,SubredditName,IsSubscribed
0,up,NaN,NaN,NaN,unexpectedMontyPython,True
1,up,197.0,2.0,Meme,ProgrammerHumor,True
2,up,11976.0,902.0,Meme,TheLastAirbender,True
3,up,158.0,9.0,NaN,risa,True
4,up,NaN,NaN,NaN,unexpectedMontyPython,True
...,...,...,...,...,...,...
1217,up,9440.0,711.0,NaN,gaming,True
1218,none,9.0,0.0,NaN,ProgrammerHumor,True
1219,up,9.0,2.0,NaN,seinfeld,True
1220,up,188.0,6.0,Meme,ProgrammerHumor,True


## Comment Votes  

Comment votes is almost identical to the post votes data except that this includes the information about comments that I have voted instead of posts. Also through Reddit API it holds the score (or the net number of upvotes) instead of seperate counts of upvotes and downvotes.  

I will perform the same cleaning as the post votes data: Remove the ID, get the subreddit name from the URL, my vote and I will compare the sub to the subscribed subs data and get wheter or not I am subscribed to that subreddit currently.  

Also, note that some of the comments or their posts are inaccesible due to different reasons which prevents data collection through Reddit API; therefore, there are some missing values in the data.

In [11]:
# Read the file, rename the vote direction column and drop the id column.
fname = "comment_votes.csv"
comment_votes_df = pd.read_csv(data_path + fname).rename(columns={"direction": "MyVote"}).drop(columns="id")

In [12]:
# Add the new columns
comment_votes_df[["SubredditName", "IsSubscribed"]] = None

for idx in range(len(comment_votes_df["permalink"])):
    comment_vote_dict = {"SubredditName": None, "IsSubscribed": None}
        
    # Get the sub name from the link
    permalink = comment_votes_df.loc[idx, "permalink"]
    start_idx = permalink.find("r/") + 2
    stop_idx = permalink.find("/", start_idx)
    sub_name = permalink[start_idx:stop_idx]

    # Add sub name to the corresponding place
    comment_votes_df.loc[idx, "SubredditName"] = sub_name

    # Check if the sub is subscribed
    comment_votes_df.loc[idx, "IsSubscribed"] = is_subscribed_to(sub_name, subreddits_df)

# Drop the permalink column
comment_votes_df = comment_votes_df.drop(columns="permalink")
# Specify the Dtypes for later use
comment_votes_df["IsSubscribed"] = comment_votes_df["IsSubscribed"].astype(dtype="bool")
comment_votes_df

,MyVote,Score,SubredditName,IsSubscribed
0,up,2.0,GenP,False
1,up,1.0,flashcarts,False
2,up,1184.0,ProgrammerHumor,True
3,up,199.0,startrek,True
4,up,5.0,startrek,True
...,...,...,...,...
255,up,36.0,CodeBullet,True
256,up,1.0,montypython,True
257,none,37.0,seinfeld,True
258,up,12.0,TheLastAirbender,True


## Posts  

The posts data is about the posts that I have created. It includes an ID, a permalink to the post, posting date, the IP that I have used, subreddit name that the post has been posted, and gildings and url data. Also the number of upvotes and downvotes through Reddit API.  

I will drop the permalink, IP, gildings, and url. Seperate the date to date and time columns, and check if I am subscribed to the sub I have posted. I will keep the IDs to compare with the comments data later on.

Also, note that some of the posts are inaccesible due to different reasons which prevents data collection through Reddit API; therefore, there are some missing values in the data.

In [13]:
# Read the file, rename the id, date, and subreddit columns and drop the permalink, ip, gildings, and url columns.
fname = "post_headers.csv"
posts_df = pd.read_csv(data_path + fname).rename(columns={"id": "ID", "date": "RawDate", "subreddit": "Subreddit"}).drop(columns=["permalink", "ip", "gildings", "url"])

In [14]:
try:
    raw_date_col = posts_df["RawDate"]  # Raw Date column

    # Add new columns
    posts_df[["IsSubscribed", "Date", "Time", "MonthName", "DayName"]] = None
    
    for idx in range(len(raw_date_col)):
        raw_date = raw_date_col.iloc[idx].replace(" UTC", "")  # Get the time in UTC time
    
        # Convert datetime to local time zone
        local_datetime = datetime.fromisoformat(raw_date).astimezone()
        datetime_dict = raw_date_to_datetime(local_datetime)

        # Insert items from datetime_dict to the new columns
        for key in datetime_dict:
            posts_df.loc[idx, key] = datetime_dict[key]
        
        # Check if subscribed
        posts_df.loc[idx, "IsSubscribed"] = is_subscribed_to(posts_df.loc[idx, "Subreddit"], subreddits_df)
    
    posts_df = posts_df.drop(columns="RawDate")
except KeyError:
    pass

# Specify the Dtypes for later use
posts_df["IsSubscribed"] = posts_df["IsSubscribed"].astype(dtype="bool")
posts_df["Date"] = pd.to_datetime(posts_df["Date"], format="%d-%m-%Y")
posts_df

,ID,Subreddit,Upvotes,Downvotes,Flair,IsSubscribed,Date,Time,MonthName,DayName
0,v7jv2a,consolerepair,3.0,0.0,NaN,True,2022-06-08,07:38+0300,June,Wednesday
1,m3jkjt,NintendoDSi,2.0,0.0,NaN,False,2021-03-12,15:13+0300,March,Friday
2,10x881w,startrek,3.0,0.0,NaN,True,2023-02-08,19:26+0300,February,Wednesday
3,126w0at,webdev,1.0,0.0,NaN,False,2023-03-30,18:58+0300,March,Thursday
4,r84lhi,flashcarts,2.0,0.0,NaN,False,2021-12-03,17:40+0300,December,Friday
5,126w1tz,webdev,1.0,0.0,NaN,False,2023-03-30,19:00+0300,March,Thursday
6,15rhuzw,montypython,58.0,1.0,NaN,True,2023-08-15,04:15+0300,August,Tuesday
7,16z0xzh,TheLastAirbender,13.0,1.0,Meme Violation,True,2023-10-03,19:25+0300,October,Tuesday
8,10vxs5t,startrek,11.0,3.0,NaN,True,2023-02-07,09:33+0300,February,Tuesday
9,zx3wgi,consolerepair,2.0,0.0,NaN,True,2022-12-28,07:54+0300,December,Wednesday


## Comments  

Similar to the data about the posts, comments data also includes an ID, a permalink to the comment, comment date, the IP that I have used, subreddit name that the post that been commented has been posted, gildings, and net score through Reddit API. It does not include a url data like posts and it holds two extra information: a link to the parent object and _if the parent is posted by me_ an ID of the parent.  

I will drop the permalink, IP, and gildings. I will seperate the date to date and time columns, check if I am subscribed to the sub I have posted, and I will check if I own the parent and the posts.  

Also, note that some of the comments or their posts are inaccesible due to different reasons which prevents data collection through Reddit API; therefore, there are some missing values in the data.

In [15]:
# Read the file, rename the id, date, and subreddit columns and drop the permalink, ip, and gildings columns.
fname = "comment_headers.csv"
comments_df = pd.read_csv(data_path + fname).rename(columns={"id": "ID", "date": "RawDate", "subreddit": "Subreddit"}).drop(columns=["permalink", "ip", "gildings"])

In [16]:
try:
    raw_date_col = comments_df["RawDate"]  # Raw Date column
    
    # Add new columns
    comments_df[["IsSubscribed", "Date", "Time", "MonthName", "DayName", "IsParentOwned", "IsPostOwned"]] = None
    
    for idx in range(len(raw_date_col)):
        raw_date = raw_date_col.iloc[idx].replace(" UTC", "")  # Get the time in UTC time
    
        # Convert datetime to local time zone
        local_datetime = datetime.fromisoformat(raw_date).astimezone()
        datetime_dict = raw_date_to_datetime(local_datetime)
    
        comments_dict = dict()
        # Check if subscribed
        comments_dict["IsSubscribed"] = is_subscribed_to(comments_df.loc[idx, "Subreddit"], subreddits_df)

        # Get the post id from the link, note that it does not have to be the parent id if it is reply to another comment
        post_link = comments_df.loc[idx, "link"]
        post_id_idx_start = post_link.find("comments/") + 9
        post_id_idx_end = post_link.find("/", post_id_idx_start)
        post_id = post_link[post_id_idx_start:post_id_idx_end]
        # Check if the post is owned by me
        comments_dict["IsPostOwned"] = (posts_df == post_id).any().any()

        # Check if parent ID exists, and if it does check if it is owned by me
        if type(comments_df.loc[idx, "parent"]) == str:
            parent_id = comments_df.loc[idx, "parent"]
            comments_dict["IsParentOwned"] = (posts_df == parent_id).any().any() or (comments_df["ID"] == parent_id).any()
        else:
            comments_dict["IsParentOwned"] = False
        
        # Combine datetime_dict and comments_dict
        comments_dict = comments_dict | datetime_dict

        # Insert items from comments_dict to the new columns
        for key in comments_dict:
            comments_df.loc[idx, key] = comments_dict[key]
    
    comments_df = comments_df.drop(columns=["RawDate", "parent", "link"])
except KeyError:
    pass

# Specify the Dtypes for later use
comments_df["IsSubscribed"] = comments_df["IsSubscribed"].astype(dtype="bool")
comments_df["IsParentOwned"] = comments_df["IsParentOwned"].astype(dtype="bool")
comments_df["Date"] = pd.to_datetime(comments_df["Date"], format="%d-%m-%Y")
comments_df

,ID,Subreddit,Score,IsSubscribed,Date,Time,MonthName,DayName,IsParentOwned,IsPostOwned
0,j7jy563,startrek,9,True,2023-02-07,09:42+0300,February,Tuesday,False,True
1,j7k0fm0,startrek,5,True,2023-02-07,10:16+0300,February,Tuesday,False,False
2,jo77dm4,veYakinEvren,1,True,2023-06-15,07:07+0300,June,Thursday,False,False
3,jr1sd8q,veYakinEvren,2,True,2023-07-07,17:33+0300,July,Friday,False,False
4,j7yb85s,gaming,1,True,2023-02-10,07:06+0300,February,Friday,False,False
...,...,...,...,...,...,...,...,...,...,...
117,jshmn5a,TheLastAirbender,5,True,2023-07-18,18:58+0300,July,Tuesday,False,False
118,jw8pa9d,tumblr,1,False,2023-08-15,04:08+0300,August,Tuesday,False,False
119,jxxbsqc,camphalfblood,3,True,2023-08-27,05:47+0300,August,Sunday,False,True
120,jvey8c7,veYakinEvren,3,True,2023-08-09,07:59+0300,August,Wednesday,False,False


# Data Visiualization

In [17]:
# Count the number of each tag from the tags column of the subreddits_df
tag_count_dict = dict()
tags_col = subreddits_df["Tags"]
for idx in range(len(tags_col)):
    tags = tags_col.iloc[idx].split(", ")
    for tag in tags:
        if tag in tag_count_dict:
            tag_count_dict[tag] += 1
        else:
            tag_count_dict[tag] = 1

# Create a bar chart of the tag counts with altair
tag_count_df = pd.DataFrame({"Tag": list(tag_count_dict.keys()), "Count": list(tag_count_dict.values())})
tag_count_df["Tag"] = tag_count_df["Tag"].astype(dtype="category")

# Create the chart
chart = alt.Chart(tag_count_df).mark_bar().encode(
    alt.X("Count:Q"),
    alt.Y("Tag:N", sort="-x"),
    color=alt.value("#1f77b4"),
    tooltip=["Count:Q"],  # Show the count when hovering over the bar
).properties(
    # Set the size of the chart
    width=600,
    height=400,
)

chart.configure(
    padding={"left": 15, "right": 15, "top": 15, "bottom": 15},
).configure_view(
    stroke=None,  # Remove the border
).configure_axisX(
    labelFontSize=14,
    titleFontSize=18,
    tickCount=tag_count_df["Count"].max() // 2,  # Set the number of ticks to half of the max count, so that the ticks are increasing by 2
    grid=False,  # Remove the grid
    domainWidth=2,  # Set the width of the axis line
    domainColor="#000"  # Set the color of the axis line
).configure_axisY(
    labelFontSize=14,
    titleFontSize=18,
    domain=False,  # Remove the axis line
)

alt.Chart(...)